In [25]:
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt

In [26]:
import plotly.io as pio
pio.renderers.default = 'chrome'

In [27]:
# This is a destination x date matrix for Manhattan residents to NYC metro municipalities 
#("subdivision-place" geography - "subpl")

df_data = pd.read_csv('../data/dest_mnxsubpl_daily_01-08-2020.csv')
df_data.head(10)

,SubPlace,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,1/7/20,1/8/20,1/9/20,...,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20
0,U0900104720,6.228492,0.000000,1.577909,4.752400,1.568923,0.000000,0.000000,0.000000,0.000000,...,12.120478,4.150842,2.058842,4.115989,0.000000,0.000000,2.042108,8.432770,6.219679,0.000000
1,U0900108070,63.842045,44.492468,52.071006,44.355733,42.360915,38.022814,52.772136,45.498863,48.755912,...,66.662627,58.111782,41.176834,22.637937,30.306704,30.231574,53.094815,52.704811,58.050338,21.898609
2,U0900108980,12.456984,9.534100,7.889546,12.673067,3.137846,1.584284,1.599156,4.874878,4.875591,...,16.160637,4.150842,6.176525,2.057994,6.061341,4.030877,4.084217,10.540962,14.512584,10.949305
3,U0900118500,29.585338,19.068201,28.402367,34.850933,43.929838,17.427123,35.181424,17.874553,16.251971,...,64.642547,72.639728,43.235676,34.985903,20.204469,20.154383,18.378974,29.514694,35.244848,12.774189
4,U0900118850,32.699584,23.835251,29.980276,44.355733,21.964919,11.089987,15.991556,17.874553,21.127562,...,26.261035,29.055891,10.294208,14.405960,12.122682,8.061753,8.168433,16.865539,16.585811,7.299536
5,U0900123890,1.557123,3.178033,4.733728,4.752400,4.706768,0.000000,0.000000,0.000000,0.000000,...,10.100398,10.377104,8.235367,6.173983,6.061341,6.046315,8.168433,12.649155,10.366132,10.949305
6,U0900126620,57.613553,36.547384,36.291913,49.108133,34.516301,31.685678,31.983113,19.499513,34.129138,...,44.441751,33.206733,20.588417,16.463954,28.286257,32.247012,53.094815,44.272041,45.610980,36.497682
7,U0900133620,105.884368,92.162970,71.005917,98.216266,73.739371,85.551331,94.350183,89.372766,78.009459,...,76.763024,76.790569,47.353359,43.217880,38.388492,66.509462,65.347465,61.137581,95.368412,40.147451
8,U0900148620,3.114246,4.767050,0.000000,3.168267,0.000000,1.584284,0.000000,0.000000,0.000000,...,6.060239,8.301683,2.058842,0.000000,6.061341,2.015438,6.126325,4.216385,8.292905,3.649768
9,U0900150580,15.571231,12.712134,23.668639,15.841333,17.258151,6.337136,7.995778,6.499838,6.500788,...,18.180716,20.754208,8.235367,8.231977,10.102235,18.138944,10.210541,18.973732,10.366132,10.949305


In [28]:
#transform table for animation, select only march - may
list_date = df_data.columns.to_list()
list_date = [x for x in list_date if x[0] == '3' or x[0] == '4' or x[0] == '5']

#pick only march 1 - march 20, because my animation won't render for more days
#remove this line if we can get 3 months to work
#0 - 20
# 21 - 41
# 41 - 61
# 61 - 81

list_date = list_date[61:81]  

df = pd.melt(df_data, id_vars=['SubPlace'],value_vars = list_date)
df = df.rename(columns={'SubPlace':'id'}) #rename for consistent geojson join
df['id'] = df['id'].str[1:] #clean the id for geojson join
df.head(10)

,id,variable,value
0,0900104720,5/1/20,0.000000
1,0900108070,5/1/20,13.692378
2,0900108980,5/1/20,2.282063
3,0900118500,5/1/20,9.128252
4,0900118850,5/1/20,4.564126
5,0900123890,5/1/20,0.000000
6,0900126620,5/1/20,18.256504
7,0900133620,5/1/20,13.692378
8,0900148620,5/1/20,0.000000
9,0900150580,5/1/20,0.000000


## Plotly

In [29]:
#read in geojson file
map_sp = json.load(open('../data/shapefiles/nycmetro_subplace.geojson','r'))

#check properties
map_sp['features'][0]['properties']

{'OBJECTID': 1,
 'STATEFP': '09',
 'COUNTYFP': '009',
 'COUSUBFP': '46940',
 'COUSUBNS': '00213459',
 'GEOID': '0900946940',
 'NAME': 'Middlebury',
 'NAMELSAD': 'Middlebury town',
 'LSAD': '43',
 'CLASSFP': 'T1',
 'MTFCC': 'G4040',
 'CNECTAFP': '720',
 'NECTAFP': '78700',
 'NCTADVFP': None,
 'FUNCSTAT': 'A',
 'ALAND': 45986818.0,
 'AWATER': 1786816.0,
 'INTPTLAT': '+41.5246912',
 'INTPTLON': '-073.1230162',
 'PLACEFP': None,
 'PLACENS': None,
 'PCICBSA': None,
 'PCINECTA': None,
 'Shape_Leng': 0.321039511792,
 'Shape_Area': 0.0051538166243,
 'ATOTAL': 47773634.0,
 'ATOTAL_mi': 18.4454,
 'geoid2': 900946940.0,
 'TotIndust': -234.0}

In [30]:
# add feature id to geojson file
for feature in map_sp['features']:
    feature['id'] = feature['properties']['GEOID']

In [31]:
map_sp['features'][0]['id']

'0900946940'

In [32]:
fig = px.choropleth_mapbox(df, geojson = map_sp,
                   locations ='id',
                   color = 'value',
                   hover_name = 'id',
                   mapbox_style="carto-positron",
                   animation_frame ='variable',
                   color_continuous_scale = "Blues",
                   labels={'value':'Trips per Device/100k'},
                   range_color = [0,150.0])

#fig[“layout”].pop(“updatemenus”)

fig.update_layout(mapbox_zoom = 7,
                 mapbox_center = {'lat':40.7, 'lon':-73.95})
fig.show()